# EDA: 500 Cities Health Data


Raw data download and description: https://www.cdc.gov/500cities/index.htm

**Intro:** The 500 Cities project is a collaboration between CDC, the Robert Wood Johnson Foundation, and the CDC Foundation. The purpose of the 500 Cities Project is to provide city- and census tract-level small area estimates for chronic disease risk factors, health outcomes, and clinical preventive service use for the largest 500 cities in the United States. These small area estimates will allow cities and local health departments to better understand the burden and geographic distribution of health-related variables in their jurisdictions, and assist them in planning public health interventions.

![picture](https://www.cdc.gov/places/about/500-cities-2016-2019/images/top-500-cities.png)




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Modules, Mount Drive, Read Data

In [ ]:
# import modules
import pandas as pd
import numpy as np
from pylab import * # simpler interface to matplotlib

In [ ]:
# copy and paste your code
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# update to your path!
# navigate on the left
# df = pd.read_csv('/content/drive/My Drive/Fall 2020 Materials/Prep/Module1_PythonDataAnalysis/2_WelcomeToPythonPt2/data/500Cities_clean.csv')
df = pd.read_csv('https://drive.google.com/uc?export=download&id=182yTU_RqX9cUcyP3MnUuaFumofNbigmP')

df.shape
df.head()

,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,Geolocation
0,CA,Folsom,624638,72203,7.5,16.9,21.8,25.7,64.8,5.8,8.6,4.1,64.7,78.1,76.6,4.1,37.1,33.3,12.2,74.7,6.7,29.1,2.1,14.3,80.4,9.9,23.8,84.3,8.9,33.9,1.9,6.8,"(38.67504943280, -121.147605753)"
1,FL,Largo,1239425,77648,19.6,30.6,16.9,36.1,81.0,9.0,7.9,9.8,77.5,80.2,64.6,10.0,33.7,33.2,20.7,58.6,12.1,39.0,3.7,31.0,75.7,13.1,28.3,77.1,15.4,37.7,4.5,18.3,"(27.90909077340, -82.7714203383)"
2,CA,Berkeley,606000,112580,7.7,15.1,19.6,20.9,68.2,4.9,8.8,3.7,64.7,70.0,75.4,3.7,38.2,36.6,11.2,70.0,6.5,27.1,2.1,14.2,81.5,10.9,18.5,83.2,8.2,32.2,1.9,6.7,"(37.87256787650, -122.274907975)"
3,CA,Napa,650258,76915,12.3,20.7,19.2,28.1,70.2,6.5,8.9,5.8,63.8,75.4,69.3,5.9,37.9,30.3,14.5,70.2,8.9,34.1,2.8,19.8,76.7,12.0,24.0,83.9,12.0,32.7,2.8,11.2,"(38.29804246490, -122.301093331)"
4,FL,Sunrise,1269700,84439,22.8,22.8,16.3,33.3,76.7,6.5,8.0,6.7,77.7,78.7,59.7,7.0,30.5,26.2,16.5,61.0,12.1,37.1,3.2,29.5,82.5,12.7,28.1,81.3,13.3,38.1,3.7,16.2,"(26.15468783030, -80.2998411020)"


In [ ]:
# data dictionary
dataDict = pd.read_excel('/content/drive/My Drive/Fall 2020 Materials/Prep/Module1_PythonDataAnalysis/2_WelcomeToPythonPt2/data/Data Dictionary_500 cities.xlsx')
# change options to show all rows
pd.set_option("display.max_rows", None, "display.max_columns", None)
dataDict

# Explore the Data
We check out the data and see that MOST data is complete, except for one column - `PAPTEST_CrudePrev` - so we will just drop this column for now.

In [ ]:
# check data types and complete values
df.info()

In [ ]:
# drop PAPTEST_CrudePrev column
df.drop(['PAPTEST_CrudePrev'], axis=1, inplace=True)
df.info() # check your work

Great! No missing values and most data is stored as numeric except for geolocation. Let's see what's going on there...

## Turn Geolocation into two columns - lat and lon
A few steps here... first, we remove the non-numeric characters '(' and ')'. Then we split the string into two columns. Then we convert to numeric and check our work.

In [ ]:
# look at the first 10 rows and scroll over - it's a string ('object')
df.head(n=10)

In [ ]:
df['Geolocation'] = df['Geolocation'].str.replace('(', '')
df['Geolocation'] = df['Geolocation'].str.replace(')', '')
df['Geolocation'] = df['Geolocation'].str.replace(',', '')
df.head() # check your work

In [ ]:
df['Geolocation'][0]

In [ ]:
# let's make a new column for latitude and longitude (the geographic coords)
new = df['Geolocation'].str.split(" ", n = 1, expand = True)
new

In [ ]:
df['lat'] = new[0] # first element of the string
df['lon'] = new[1] # second element of the string
df.head()

In [ ]:
# check column types
df.info()

# uh oh! still stored as a string!

In [ ]:
# lat to numeric
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
# lon to numeric
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

df.info()
df.head()

In [ ]:
# great! now drop the 'Geolocation' column since we don't need it
del df['Geolocation']
df.head()

# Summary Statistics/Percentiles
Since MOST of our data is numeric (except for 'StateAbbr', 'PlaceName'), we can compute numeric summary stats via .describe() from a pandas dataframe.

Right away you can compare rates and identify which variables have skewed distributions.

In [ ]:
# link: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html
df.describe()

Want more detail? Try some percentiles. With pandas, you can create summary stats right on the dataframe itself.

Now you can say things like 20% of towns have 17.9% or less of the population suffering from ARTHRITIS etc.

In [ ]:
# link: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html
# calculate percentiles for all numeric variables
df.quantile(np.arange(0,1.1,0.1)) # every 10%! change to 0.05 for every 5%

# GroupBy
Let's see how many cities there are per State using groupby our value_counts.

In [ ]:
# get a print-out of all the columns
df.columns

Using `value_counts()` from pandas is really nice - this is the count of large cities per State - and we note most of them are in California!

In [ ]:
# value counts per 'StateAbbr'
df['StateAbbr'].value_counts()

Another way we can do this is with a 'groupby' statement - 'groupby' is nice because you can make a ton of other stats per group. Full customization!

Look at how much cool stuff you can do here: https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/

In [ ]:
# for example, let's look at the rates of arthritis for each State
tmp = df.groupby(['StateAbbr'])['ARTHRITIS_CrudePrev'].mean().reset_index()
tmp # check your work!

## Sort Values

In [ ]:
# this is certainly helpful, but maybe we can sort this table
# so we can look at the biggest and largest states
# link: https://stackoverflow.com/questions/37787698/how-to-sort-pandas-dataframe-from-one-column
tmp = tmp.sort_values('ARTHRITIS_CrudePrev', ascending=True)
tmp

# we see that VT, DC and CA are the best States (lowest rates) for arthritis

In [ ]:
# how about the highest rates?
# link: https://stackoverflow.com/questions/37787698/how-to-sort-pandas-dataframe-from-one-column
tmp = tmp.sort_values('ARTHRITIS_CrudePrev', ascending=False)
tmp

# West Virginia, Ohio and Michigan and Mississippi

# Data Enrichment
Did you know you can grab .csv files from Github and tables from Wikipedia using pandas? It's awesome.

We will grab a list of 'regions and divisions' from Github and we will grab some economic data from Wikipedia then join it all together.

In [ ]:
# grab a .csv file from Github
# once you go to Github, click through to 'Raw Data'
# make sure your link has rawdata at the beggining of the URL

# link to github: https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv
# click through to raw data: https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv
regionData = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')
regionData.head()

# you will be able to join the 'State Code' from regionData and the 'StateAbbr' column from df
# and you will have enriched your df data! much more cool columns to do aggregations/tables by.

In [ ]:
# steal some economic data off of Wikipedia

# link: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_table.html
# link: https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_GDP

econData = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_GDP')[2]
econData

***SWEET!*** Now we have a ton of geographic and economic data that we can join together.

First, we can join the `regionData` to `df`, then we can join `econData` to `df` using the `Statefederal district or territory` column.

Be careful of keys - sometimes you need to spend a little time cleaning up text data - part of the joys of being a data analyst or data scientist!

In [ ]:
print(df.columns)
print(regionData.columns)

In [ ]:
print(df.shape)
df.head()

In [ ]:
print(regionData.shape)
regionData.head()

Wow - we are starting to get some really cool features for analysis/exploration...

In [ ]:
# join regionData to df
# link: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
df = df.merge(regionData, left_on='StateAbbr', right_on='State Code')

# check your work
df.info()
df.head()

Keep enriching! Now add the econ data.

In [ ]:
print(df.columns)
print(econData.columns)

In [ ]:
# this is df data
print(df.shape)
df.head()

In [ ]:
# this is econData
print(econData.shape)
econData.head()

In [ ]:
# now we join econData to our newly enriched 'df'
# join regionData to df
df = df.merge(econData, left_on='State', right_on='Statefederal district or territory')

# check your work
df.info()
df.head()

## Column Names/ Repetitive Columns
The data is complete, but some of our headers are clunky. Let's rename them.

In [ ]:
# first, let's drop any columns that have 100% duplicate information
del df['Region_y']

# and let's rename 'Region_y' to just 'Region'
df.rename(columns={'Region_x':'Region'}, inplace=True) # don't forget this!
df.info() # show some good info on non-null values and dtypes
df.head() # shows first few rows

It looks like `2019[note 1]` could be renamed `2019Population`; `Statefederal district or territory` looks like a repeat of `States`; and columns `Rank`, `2019Population`, `% of Nation` and `GDP` are numbers not objects (strings). So we will convert them.

In [ ]:
df.columns

In [ ]:
# delete a column
del df['Statefederal district or territory']

In [ ]:
# rename one more column
df.rename(columns={'2019[note 1]':'Pop2019'}, inplace=True)
print(df.shape)
df.head()

## Converting data type: to_numeric()

In [ ]:
# and convert those few columns to numeric
# link: https://stackoverflow.com/questions/36814100/pandas-to-numeric-for-multiple-columns
cols = ['Rank',	'Pop2019',	'% of Nation',	'GDP per capita']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# check your work
df.info()
df.head()


# All Done!
You have made some incredible data for analysis (which you will use in a future optimization model). Now you can make some interesting plots and tables on this dataset.

In [ ]:
# save a copy to your Google Drive

# update with a path to your Google Drive
df.to_csv('/content/drive/My Drive/Fall 2020 Materials/Prep/Module1_PythonDataAnalysis/2_WelcomeToPythonPt2/data/500Cities_enriched.csv')

# you can go into your Google Drive and verify that it's there